In [2]:
from sqlalchemy import create_engine

In [3]:
user = "schedulin"
passw = "MySQLIsFun"
host = "35.231.228.133"
database = "schedulin"

In [4]:
db = create_engine(
    'mysql+pymysql://{0}:{1}@{2}/{3}' \
        .format(user, passw, host, database), \
    connect_args = {'connect_timeout': 10})
conn = db.connect()

In [36]:
def query_db(query):
    results = conn.execute(query)
    print(list(results.keys()))
    print('----------------------------')
    for r in results.fetchall():
        print(r)

In [37]:
query = """
    SHOW TABLES;
"""
query_db(query)

['Tables_in_schedulin']
----------------------------
('reservation',)
('resource',)
('timetable',)
('user',)


In [38]:
query = """
    SELECT 
        COUNT(*) 
    FROM reservation;
"""
query_db(query)

['COUNT(*)']
----------------------------
(69151,)


In [39]:
query = """
    SELECT 
        DISTINCT 
            type 
    FROM resource;
"""
query_db(query)

['type']
----------------------------
('PADEL',)
('PISCINA',)
('GIMNASIO',)
('SALA COMUN',)


In [40]:
query = """
    SELECT 
        type,
        price    
    FROM resource
    WHERE price = (
        SELECT 
            MAX(price) AS max
        FROM resource
        GROUP BY type
        ORDER BY MAX(price) DESC
        LIMIT 1);
"""
query_db(query)

['type', 'price']
----------------------------
('GIMNASIO', Decimal('5.00'))
('SALA COMUN', Decimal('5.00'))


In [41]:
query = """
    SELECT 
        MIN(start_time), 
        MAX(end_time) 
    FROM timetable AS t 
    LEFT JOIN resource AS r 
    ON t.id = r.timetable_id 
    WHERE r.type = 'PADEL';
"""
query_db(query)

['MIN(start_time)', 'MAX(end_time)']
----------------------------
('09:00', '22:30')


In [43]:
query = """
    SELECT 
        DISTINCT
            TIMEDIFF(
            STR_TO_DATE(end_time,'%%H:%%i'), 
            STR_TO_DATE(start_time,'%%H:%%i')) AS TIMEDIFF
    FROM timetable AS t 
    LEFT JOIN resource AS r 
    ON t.id = r.timetable_id 
    WHERE r.type = 'PADEL';
"""
query_db(query)

['TIMEDIFF']
----------------------------
(datetime.timedelta(seconds=5400),)


In [44]:
query = """
    SELECT
        AVG(hours_in_advance) AS hours
    FROM resource;
"""
query_db(query)

['hours']
----------------------------
(Decimal('6.5714'),)


In [46]:
query = """
    SELECT
        AVG(price) AS price
    FROM resource;
"""
query_db(query)

['price']
----------------------------
(Decimal('1.857143'),)


In [47]:
query = """
    SELECT
    --    COUNT(DISTINCT(email)) AS users
        COUNT(ID) AS users
    FROM user;
"""
query_db(query)

['users']
----------------------------
(380,)


In [48]:
query = """
    SELECT
        COUNT(ID) AS users
    FROM user
    WHERE status = 'INACTIVE';
"""
query_db(query)

['users']
----------------------------
(45,)


In [49]:
query = """
    SELECT 
        email
    FROM user
    WHERE deletion_date IS NOT NULL;
"""
query_db(query)

['email']
----------------------------
('elizabeth.fletcher@mail.com',)
('jorge.santos@mail.com',)
('sandra.evans@mail.com',)
('connor.harris@mail.com',)
('gina.owens@mail.com',)
('michael.bartlett@mail.com',)
('christopher.sanders@mail.com',)
('susan.pace@mail.com',)
('jennifer.hayes@mail.com',)
('jim.davis@mail.com',)
('crystal.grant@mail.com',)
('joseph.steele@mail.com',)
('amanda.pope@mail.com',)
('kevin.clark@mail.com',)


In [50]:
query = """
    SELECT
        email
    FROM user
    WHERE deletion_date IS NOT NULL
        AND status = 'ACTIVE';
"""
query_db(query)

['email']
----------------------------


In [51]:
query = """
    SELECT
        DATE(deletion_date),
        COUNT(deletion_date) AS total
    FROM user
    WHERE deletion_date IS NOT NULL
    GROUP BY DATE(deletion_date)
    LIMIT 1;
"""
query_db(query)

['DATE(deletion_date)', 'total']
----------------------------
(datetime.date(2018, 7, 11), 6)


In [52]:
query = """
    SELECT
        u.id, 
        u.deletion_date, 
        r.date
    FROM (
        SELECT id, DATE(deletion_date) AS deletion_date
        FROM user
        WHERE deletion_date IS NOT NULL) AS u
    INNER JOIN (
        SELECT *
        FROM reservation) AS r
    ON u.id = r.user_id
        AND u.deletion_date < r.date
    LIMIT 10;
"""
query_db(query)

['id', 'deletion_date', 'date']
----------------------------
(7, datetime.date(2018, 7, 11), datetime.date(2022, 9, 18))
(7, datetime.date(2018, 7, 11), datetime.date(2022, 5, 11))
(7, datetime.date(2018, 7, 11), datetime.date(2018, 10, 23))
(7, datetime.date(2018, 7, 11), datetime.date(2020, 1, 29))
(7, datetime.date(2018, 7, 11), datetime.date(2019, 1, 21))
(7, datetime.date(2018, 7, 11), datetime.date(2020, 8, 30))
(7, datetime.date(2018, 7, 11), datetime.date(2021, 12, 30))
(7, datetime.date(2018, 7, 11), datetime.date(2019, 3, 8))
(7, datetime.date(2018, 7, 11), datetime.date(2020, 1, 5))
(7, datetime.date(2018, 7, 11), datetime.date(2022, 8, 4))


In [53]:
query = """
    SELECT 
        user_id,
        total
    FROM(
    SELECT 
        user_id,
        COUNT(user_id) AS total
    FROM reservation
    GROUP BY user_id) as t
    WHERE total = (
        SELECT
            MAX(total) AS MAX
        FROM (
            SELECT 
                user_id,
                COUNT(user_id) AS total
            FROM reservation
            GROUP BY user_id
            ORDER BY total DESC
            LIMIT 1) AS m);
"""
query_db(query)

['user_id', 'total']
----------------------------
(119, 251)


In [54]:
query = """
    SELECT
        user_id,
        COUNT(user_id) AS total
    FROM reservation
    WHERE YEAR(date) = 2022
    GROUP BY user_id
    ORDER BY total DESC
    LIMIT 1;
"""
query_db(query)

['user_id', 'total']
----------------------------
(176, 55)


In [55]:
query = """
    SELECT 
        only_name, 
        COUNT(only_name) AS total 
    FROM (
        SELECT 
            name, 
            TRIM(SUBSTRING_INDEX(name, " ", -1)) AS only_name 
        FROM user) AS m 
    GROUP BY only_name 
    ORDER BY total DESC
    LIMIT 10;
"""
query_db(query)

['only_name', 'total']
----------------------------
('MD', 8)
('Brown', 7)
('Smith', 7)
('Johnson', 6)
('Jones', 6)
('Miller', 5)
('Harris', 4)
('Wilson', 4)
('Stewart', 4)
('Jackson', 4)


In [58]:
query = """
    SELECT
        status, 
        (COUNT(*) / (
            SELECT COUNT(*)
            FROM reservation)) * 100 AS 'perc'
    FROM reservation 
    GROUP BY status;
"""
query_db(query)

['status', 'perc']
----------------------------
('CONFIRMED', Decimal('92.5569'))
('CANCELLED', Decimal('5.4461'))
('EXECUTED', Decimal('1.9971'))


In [59]:
query = """
    SELECT
        type,
        TOTAL
    FROM (
        SELECT 
            RE.type,
            COUNT(*) AS TOTAL
        FROM (
            SELECT 
                resource_id
            FROM reservation) AS R
        LEFT JOIN (
            SELECT
                id, type
            FROM resource) AS RE
        ON R.resource_id = RE.id
        GROUP BY RE.type) AS T
    WHERE TOTAL = (
        SELECT 
            MAX(TOTAL)
        FROM (
            SELECT 
                RE.type,
                COUNT(*) AS TOTAL
            FROM (
                SELECT 
                    resource_id
                FROM reservation) AS R
            LEFT JOIN (
                SELECT
                    id, type
                FROM resource) AS RE
            ON R.resource_id = RE.id
            GROUP BY RE.type) AS M);
"""
query_db(query)

['type', 'TOTAL']
----------------------------
('PADEL', 44292)


In [60]:
query = """
    SELECT
        type,
        TOTAL
    FROM (
        SELECT 
            RE.type,
            COUNT(*) AS TOTAL
        FROM (
            SELECT 
                resource_id
            FROM reservation) AS R
        LEFT JOIN (
            SELECT
                id, type
            FROM resource) AS RE
        ON R.resource_id = RE.id
        GROUP BY RE.type) AS T
    WHERE TOTAL = (
            SELECT 
                COUNT(*) AS TOTAL
            FROM (
                SELECT 
                    resource_id
                FROM reservation) AS R
            LEFT JOIN (
                SELECT
                    id, type
                FROM resource) AS RE
            ON R.resource_id = RE.id
            GROUP BY RE.type
            LIMIT 1);
"""
query_db(query)

['type', 'TOTAL']
----------------------------
('PADEL', 44292)


In [61]:
query = """
    SELECT
        start_time, end_time,
        COUNT(start_time) AS TOTAL
    FROM (
        SELECT id, resource_id
        FROM reservation) AS RE
    LEFT JOIN (
        SELECT id, timetable_id, type
        FROM resource) AS R
    ON RE.resource_id = R.id
    LEFT JOIN (
        SELECT id, start_time, end_time
        FROM timetable) AS T
    ON T.id = R.timetable_id
    GROUP BY start_time, end_time
    ORDER BY TOTAL DESC;
"""
query_db(query)

['start_time', 'end_time', 'TOTAL']
----------------------------
('10:00', '16:00', 10096)
('16:00', '22:00', 9746)
('18:00', '19:30', 5041)
('10:00', '22:00', 5017)
('09:00', '10:30', 4983)
('16:30', '18:00', 4946)
('21:00', '22:30', 4929)
('15:00', '16:30', 4914)
('13:30', '15:00', 4896)
('10:30', '12:00', 4894)
('19:30', '21:00', 4860)
('12:00', '13:30', 4829)


In [62]:
query = """
    SELECT 
       AVG(TOTAL) AS AVG
    FROM (
        SELECT
            date,
            COUNT(*) AS TOTAL
        FROM reservation
        GROUP BY date) AS R;
"""
query_db(query)

['AVG']
----------------------------
(Decimal('18.9299'),)


In [63]:
query = """
    SELECT
        G,
        COUNT(*) AS TOTAL
    FROM (
        SELECT 
            CASE
                WHEN STR_TO_DATE(start_time,'%%H:%%i')
                    >= STR_TO_DATE('16:00','%%H:%%i') THEN 'EVENING'
                ELSE 'MORNING'
            END AS 'G'
        FROM (
            SELECT id, resource_id
            FROM reservation) AS RE
        LEFT JOIN (
            SELECT id, timetable_id, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        LEFT JOIN (
            SELECT id, start_time
            FROM timetable) AS T
        ON T.id = R.timetable_id) AS EM
    GROUP BY G
    ORDER BY TOTAL DESC;
"""
query_db(query)

['G', 'TOTAL']
----------------------------
('MORNING', 39629)
('EVENING', 29522)


In [64]:
query = """
    SELECT
        HAS_PROBLEMS,
        COUNT(*) AS TOTAL
    FROM (
        SELECT 
            CASE
                WHEN max_pax < num_pax THEN 1
                ELSE 0
            END AS 'HAS_PROBLEMS'
        FROM (
            SELECT id, resource_id, num_pax
            FROM reservation) AS RE
        LEFT JOIN (
            SELECT id, timetable_id, max_pax
            FROM resource) AS R
        ON RE.resource_id = R.id) AS M
        GROUP BY HAS_PROBLEMS
        ORDER BY TOTAL DESC;
"""
query_db(query)

['HAS_PROBLEMS', 'TOTAL']
----------------------------
(0, 52092)
(1, 17059)


In [66]:
query = """
    SELECT
        user_id,
        date,
        start_time,
        GROUP_CONCAT(DISTINCT type) AS 'types_arr',
        COUNT(*) AS TOTAL
    FROM (
        SELECT user_id, date, start_time, type
        FROM (
            SELECT id, resource_id, user_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
        LEFT JOIN (
            SELECT id, timetable_id, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        LEFT JOIN (
            SELECT id, start_time, end_time
            FROM timetable) AS T
        ON T.id = R.timetable_id) AS M
    GROUP BY user_id, date, start_time
    HAVING TOTAL > 1
    LIMIT 10;
"""
query_db(query)

['user_id', 'date', 'start_time', 'types_arr', 'TOTAL']
----------------------------
(1, datetime.date(2013, 9, 26), '12:00', 'PADEL', 2)
(3, datetime.date(2021, 6, 8), '13:30', 'PADEL', 2)
(3, datetime.date(2021, 6, 30), '16:00', 'GIMNASIO', 2)
(4, datetime.date(2021, 5, 18), '10:00', 'SALA COMUN', 2)
(7, datetime.date(2022, 4, 18), '16:00', 'GIMNASIO,PISCINA', 2)
(12, datetime.date(2014, 8, 29), '21:00', 'PADEL', 2)
(13, datetime.date(2016, 7, 31), '18:00', 'PADEL', 2)
(13, datetime.date(2019, 12, 23), '15:00', 'PADEL', 2)
(15, datetime.date(2019, 9, 5), '15:00', 'PADEL', 2)
(15, datetime.date(2020, 1, 28), '10:00', 'GIMNASIO,SALA COMUN', 2)


In [67]:
query = """
        SELECT user_id, date, start_time, end_time, type
        FROM (
            SELECT id, resource_id, user_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')
                AND user_id = 377
                AND date = '2014-11-19') AS RE
        LEFT JOIN (
            SELECT id, timetable_id, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        LEFT JOIN (
            SELECT id, start_time, end_time
            FROM timetable) AS T
        ON T.id = R.timetable_id
        ORDER BY start_time ASC
"""
query_db(query)

['user_id', 'date', 'start_time', 'end_time', 'type']
----------------------------
(377, datetime.date(2014, 11, 19), '10:00', '16:00', 'PISCINA')
(377, datetime.date(2014, 11, 19), '13:30', '15:00', 'PADEL')


In [68]:
# https://www.mysqltutorial.org/mysql-window-functions/mysql-lag-function/

query = """
    SELECT *
    FROM (
        SELECT  user_id, date, start_time, end_time, post_start_time,
            CASE 
                WHEN end_time > post_start_time THEN 1
                ELSE 0
            END HAS_DUP
        FROM (
            SELECT user_id, date, start_time, end_time,
                LAG(start_time, 1) OVER (
                    PARTITION BY user_id, date
                    ORDER BY start_time DESC) AS post_start_time
            FROM (
                SELECT id, resource_id, user_id, date
                FROM reservation
                WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
            LEFT JOIN (
                SELECT id, timetable_id, type
                FROM resource) AS R
            ON RE.resource_id = R.id
            LEFT JOIN (
                SELECT id, start_time, end_time
                FROM timetable) AS T
            ON T.id = R.timetable_id
            ORDER BY user_id DESC, date DESC, start_time DESC) AS M
        WHERE post_start_time IS NOT NULL) AS X
    WHERE HAS_DUP = 1
    LIMIT 10;
"""
query_db(query)

['user_id', 'date', 'start_time', 'end_time', 'post_start_time', 'HAS_DUP']
----------------------------
(380, datetime.date(2022, 5, 20), '10:00', '16:00', '13:30', 1)
(380, datetime.date(2022, 4, 18), '10:00', '22:00', '19:30', 1)
(380, datetime.date(2021, 2, 1), '10:00', '22:00', '10:00', 1)
(379, datetime.date(2022, 4, 20), '10:00', '22:00', '10:00', 1)
(379, datetime.date(2022, 1, 20), '09:00', '10:30', '10:00', 1)
(379, datetime.date(2021, 6, 29), '10:00', '16:00', '12:00', 1)
(378, datetime.date(2022, 3, 10), '10:00', '16:00', '12:00', 1)
(377, datetime.date(2021, 5, 28), '15:00', '16:30', '16:00', 1)
(377, datetime.date(2014, 11, 19), '10:00', '16:00', '13:30', 1)
(376, datetime.date(2022, 11, 3), '10:00', '22:00', '16:30', 1)


In [69]:
query = """

            SELECT user_id, SUM(price) AS TOTAL
            FROM (
                SELECT id, resource_id, user_id, date
                FROM reservation
                WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
            LEFT JOIN (
                SELECT id, price
                FROM resource) AS R
            ON RE.resource_id = R.id
            GROUP BY user_id
            ORDER BY TOTAL DESC
            LIMIT 10
"""
query_db(query)

['user_id', 'TOTAL']
----------------------------
(119, Decimal('471.25'))
(151, Decimal('466.00'))
(176, Decimal('459.25'))
(125, Decimal('446.25'))
(144, Decimal('446.00'))
(128, Decimal('445.75'))
(167, Decimal('445.25'))
(172, Decimal('444.50'))
(180, Decimal('435.25'))
(109, Decimal('431.00'))


In [70]:
query = """

            SELECT user_id, SUM(price) AS TOTAL
            FROM (
                SELECT id, resource_id, user_id, date
                FROM reservation
                WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
            LEFT JOIN (
                SELECT id, price
                FROM resource) AS R
            ON RE.resource_id = R.id
            GROUP BY user_id
            ORDER BY TOTAL ASC
            LIMIT 10
"""
query_db(query)

['user_id', 'TOTAL']
----------------------------
(244, Decimal('212.00'))
(209, Decimal('223.00'))
(297, Decimal('225.25'))
(263, Decimal('226.75'))
(291, Decimal('235.00'))
(186, Decimal('237.25'))
(312, Decimal('238.00'))
(301, Decimal('238.00'))
(289, Decimal('241.50'))
(86, Decimal('241.75'))


In [71]:
query = """
    SELECT id, timetable_id, price, max_pax
    FROM resource
    WHERE type = 'GIMNASIO'
"""
query_db(query)

['id', 'timetable_id', 'price', 'max_pax']
----------------------------
(12, 11, Decimal('3.00'), 6)
(13, 12, Decimal('5.00'), 6)


In [72]:
query = """
    SELECT id, start_time, end_time
    FROM timetable
    WHERE id = 10
"""
query_db(query)

['id', 'start_time', 'end_time']
----------------------------
(10, '10:00', '22:00')


In [75]:
query = """
SELECT Y.TOTAL + Z.TOTAL AS 'total'
FROM (
    SELECT SUM(price) * 365 AS TOTAL
    FROM (
        SELECT id, price
        FROM resource
        WHERE type IN ('PADEL', 'SALA COMUN')) AS R) AS Y
LEFT JOIN (
    SELECT SUM(price * max_pax) * 365 AS TOTAL
    FROM (
        SELECT id, price, max_pax
        FROM resource
        WHERE type = 'GIMNASIO') AS P) AS Z
ON 1 = 1;
"""
query_db(query)

['total']
----------------------------
(Decimal('24090.00'),)


In [76]:
query = """
SELECT SUM(TOTAL) AS 'total'
FROM (
    SELECT SUM(price) * 365 AS TOTAL
    FROM (
        SELECT id, price
        FROM resource
        WHERE type IN ('PADEL', 'SALA COMUN')) AS R
    UNION
    SELECT SUM(price * max_pax) * 365 AS TOTAL
    FROM (
        SELECT id, price, max_pax
        FROM resource
        WHERE type = 'GIMNASIO') AS P) AS Z;
"""
query_db(query)

['total']
----------------------------
(Decimal('24090.00'),)


In [77]:
query = """
SELECT
    AVG(TOTAL) AS 'total'
FROM (
    SELECT
        YEAR(date) AS YEAR, 
        SUM(price) AS TOTAL
    FROM (

        SELECT date, price
        FROM (
            SELECT id, resource_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
        LEFT JOIN (
            SELECT id, price
            FROM resource) AS R
        ON RE.resource_id = R.id) AS R
    GROUP BY YEAR(date)) AS D;
"""
query_db(query)

['total']
----------------------------
(Decimal('11084.545455'),)


In [79]:
query = """
        SELECT 
            date, type, SUM(price) AS 'price', COUNT(date) AS 'total'
        FROM (
            SELECT id, resource_id, date
            FROM reservation
            WHERE status IN ('CONFIRMED', 'EXECUTED')) AS RE
        LEFT JOIN (
            SELECT id, price, type
            FROM resource) AS R
        ON RE.resource_id = R.id
        GROUP BY date, type
        ORDER BY date DESC, type DESC
        LIMIT 10;
"""
query_db(query)

['date', 'type', 'price', 'total']
----------------------------
(datetime.date(2022, 11, 4), 'SALA COMUN', Decimal('10.00'), 2)
(datetime.date(2022, 11, 4), 'PISCINA', Decimal('0.00'), 8)
(datetime.date(2022, 11, 4), 'PADEL', Decimal('28.00'), 21)
(datetime.date(2022, 11, 4), 'GIMNASIO', Decimal('39.00'), 9)
(datetime.date(2022, 11, 3), 'SALA COMUN', Decimal('5.00'), 1)
(datetime.date(2022, 11, 3), 'PISCINA', Decimal('0.00'), 1)
(datetime.date(2022, 11, 3), 'PADEL', Decimal('42.00'), 36)
(datetime.date(2022, 11, 3), 'GIMNASIO', Decimal('37.00'), 9)
(datetime.date(2022, 11, 2), 'SALA COMUN', Decimal('10.00'), 2)
(datetime.date(2022, 11, 2), 'PISCINA', Decimal('0.00'), 4)
